# Billboard CSV Cleaning

Clean billboard dataset for BigQuery - remove multi-line lyrics and normalize text

In [11]:
library(tidyverse)
library(readr)

In [12]:
# load data
input <- '../../data/billboard_24years_lyrics_spotify.csv'
output <- '../../data/cleaned/billboard_24years_lyrics_spotify_bigquery.csv'

df <- read_csv(input, na=c("", "NA"))
sprintf("Loaded %d rows", nrow(df))

Rows: 3397 Columns: 26
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (11): song, band_singer, songurl, titletext, url, lyrics, uri, type, id,...
dbl (15): ranking, year, danceability, energy, key, loudness, mode, speechin...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Loaded 3397 rows"

In [13]:
glimpse(df)

Rows: 3,397
Columns: 26
$ ranking          <dbl> 1, 2, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ~
$ song             <chr> "Breathe", "Smooth", "Smooth", "Maria Maria", "Maria ~
$ band_singer      <chr> "Faith Hill", "Santana", "Rob Thomas", "Santana", "Th~
$ songurl          <chr> "/wiki/Breathe_(Faith_Hill_song)", "/wiki/Smooth_(San~
$ titletext        <chr> "Breathe", "Smooth", "Smooth", "Maria Maria", "Maria ~
$ url              <chr> "/wiki/Faith_Hill", "/wiki/Santana_(band)", "/wiki/Ro~
$ year             <dbl> 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,~
$ lyrics           <chr> "I can feel the magic floating in the air\nBeing with~
$ uri              <chr> "spotify:track:3y4LxiYMgDl4RethdzpmNe", "spotify:trac~
$ danceability     <dbl> 0.529, 0.609, 0.590, 0.777, NA, 0.725, 0.636, NA, 0.6~
$ energy           <dbl> 0.496, 0.923, 0.637, 0.601, NA, 0.487, 0.761, NA, 0.4~
$ key              <dbl> 7, 9, 9, 2, NA, 8, 3, NA, 9, 1, 6, 11, 2, 6, NA, 2, 1~
$ loudness      

In [14]:
# check spotify features
spotify_cols <- c('danceability', 'energy', 'tempo', 'duration_ms', 'valence')
sapply(df[spotify_cols], function(x) sum(is.na(x)))

danceability       energy        tempo  duration_ms      valence 
        2911         2911         2911         2911         2911

In [15]:
# clean text fields
df2 <- df %>% 
  mutate(across(where(is.character), function(x) {
    x <- str_replace_all(x, "\\n", " ")
    x <- str_replace_all(x, "\\r", "")
    x <- str_replace_all(x, "\\s+", " ")
    str_trim(x)
  }))

sprintf("Cleaned %d rows", nrow(df2))

[1] "Cleaned 3397 rows"

In [16]:
# verify
check <- df2 %>% filter(str_detect(lyrics, "\\n"))
if(nrow(check) == 0) {
  print("✓ No newlines")
}

[1] "<U+2713> No newlines"


In [17]:
# export
dir.create("../../data/cleaned", showWarnings=F, recursive=T)
write_csv(df2, output, na="", quote="all", eol="\n")
sprintf("Wrote: %s", output)

[1] "Wrote: ../../data/cleaned/billboard_24years_lyrics_spotify_bigquery.csv"

In [18]:
# check file
lines <- length(readLines(output))
expected <- nrow(df2) + 1
sprintf("Lines: %d (expected %d)", lines, expected)

[1] "Lines: 3398 (expected 3398)"

In [19]:
# preview final output
df2 %>% 
  select(song, band_singer, year, danceability, energy, tempo) %>% 
  head(10)

song,band_singer,year,danceability,energy,tempo
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Breathe,Faith Hill,2000,0.529,0.496,136.859
Smooth,Santana,2000,0.609,0.923,115.996
Smooth,Rob Thomas,2000,0.590,0.637,115.983
Maria Maria,Santana,2000,0.777,0.601,97.911
Maria Maria,The Product G&B,2000,NA,NA,NA
I Wanna Know,Joe,2000,0.725,0.487,136.086
Everything You Want,Vertical Horizon,2000,0.636,0.761,93.896
Say My Name,Destiny's Child,2000,NA,NA,NA
I Knew I Loved You,Savage Garden,2000,0.610,0.497,85.031


In [20]:
# lyrics preview
df2 %>% 
  select(song, band_singer, lyrics) %>% 
  mutate(lyrics <- str_trunc(lyrics, 80)) %>% 
  head(5)

song,band_singer,lyrics,"lyrics <- str_trunc(lyrics, 80)"
<chr>,<chr>,<chr>,<chr>
Breathe,Faith Hill,"I can feel the magic floating in the air Being with you gets me that way I watch the sunlight dance across your face and I've Never been this swept away All my thoughts just seem to settle on the breeze When I'm lying wrapped up in your arms The whole world just fades away The only thing I hear Is the beating of your heart And I can feel you breathe, it's washing over me And suddenly, I'm melting into you There's nothing left to prove Baby, all we need is just to be Caught up in the touch, slow and steady rush Baby, isn't that the way that love's supposed to be? I can feel you breathe Just breathe In a way, I know my heart is waking up As all the walls come tumbling down I'm closer than I've ever felt before And I know, and you know There's no need for words right now You might also like And I can feel you breathe, it's washing over me And suddenly, I'm melting into you There's nothing left to prove Baby, all we need is just to be Caught up in the touch, slow and steady rush Baby, isn't that the way that love's supposed to be? I can feel you breathe Just breathe Caught up in the touch The slow and steady rush Baby, isn't that the way that love's supposed to be? I can feel you breathe Just breathe I can feel the magic floating in the air Being with you gets me that way7Embed",I can feel the magic floating in the air Being with you gets me that way I wa...
Smooth,Santana,"Man, it's a hot one Like seven inches from the midday sun Well, I hear you whisper and the words melt everyone But you stay so cool My mu<U+00F1>equita, my Spanish Harlem Mona Lisa You're my reason for reason, the step in my groove, yeah And if you say, ""This life ain't good enough"" I would give my world to lift you up I could change my life to better suit your mood 'Cause you're so smooth And it's just like the ocean under the moon Well, it's the same as the emotion that I get from you You got the kind of lovin' that can be so smooth Gimme your heart, make it real, or else forget about it Well, I'll tell you one thing If you would leave it'd be a crying shame In every breath and every word I hear your name callin' me out Out from the barrio, you hear my rhythm on the radio You feel the turning of the world so soft and slow Turnin' you round and round You might also like And if you say, ""This life ain't good enough"" I would give my world to lift you up I could change my life to better suit your mood 'Cause you're so smooth Oh, and it's just like the ocean under the moon Well, it's the same as the emotion that I get from you You got the kind of loving that can be so smooth, yeah Gimme your heart, make it real, or else forget about it, yeah And it's just like the ocean under the moon Well, it's the same as the emotion that I get from you You got the kind of loving that can be so smooth, yeah Gimme your heart, make it real, or else forget about it, damn Or else forget about it Oh, or else forget about it Oh, let's don't forget about it (Gimme your heart, make it real) Or else forget about it, hey Let's don't forget about it, no, oh, no, oh Let's don't forget about it, no, no, no, oh Let's don't forget about it, hey, no, no, oh Let's don't forget about it, hey, hey, hey24Embed","Man, it's a hot one Like seven inches from the midday sun Well, I hear you wh..."
Smooth,Rob Thomas,"Man, it's a hot one Like seven inches from the midday sun Well, I hear you whisper and the words melt everyone But you stay so cool My mu<U+00F1>equita, my Spanish Harlem Mona Lisa You're my reason for reason, the step in my groove, yeah And if you say, ""This life ain't good enough"" I would give my world to lift you up I could change my life to better suit your mood 'Cause you're so smooth And it's just like the ocean under the moon Well, it's the same as the emotion that I get from you You got the kind of lovin' that can be so smooth Gimme your heart, make it real, or else forget abou